In [ ]:
#default_exp models.core

# Model Foundations
> Core aspects to building a `SAINT` model

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#export
import torch
import torch.nn.functional as F
from torch import nn, einsum
import numpy as np
from einops import rearrange

from fastai.layers import Module

from fastcore.basics import mk_class

In [ ]:
#export
_all_ = ['AttentionType']

## Helper Functions

In [ ]:
#export
def exists(
    val # Any value
):
    "Tests if `val` is not None"
    return val is not None

In [ ]:
#hide
a = 'blah'
test_eq(exists(a), True)

In [ ]:
#hide
b = None
test_eq(exists(b), False)

In [ ]:
#export
def default(
    val, # Any value
    d # Some default for `val`
):
    "Get a default if `val` doesn't exist"
    return val if exists(val) else d

In [ ]:
#hide
a = None
d = 42
test_eq(default(a,d), 42)

In [ ]:
#export
def ff_encodings(
    x:torch.tensor, # Input
    B:torch.tensor # Projection matrix
):
    "Return sin and cosine projections of `x` @ `B`"
    x_proj = (2. * np.pi * x.unsqueeze(-1)) @ B.t()
    return torch.cat([torch.sin(x_proj), torch.cos(x_proj)], dim=-1)

## Basic Layers

In [ ]:
#export
class Residual(Module):
    def __init__(
        self,
        fn:callable, # A function to generate a residual
    ):
        "A layer that adds a residual `fn` to x"
        self.fn = fn
    
    def forward(
        self,
        x:torch.tensor, # An input
        **kwargs, # kwargs for `self.fn`
    ):
        "Applies `self.fn` to x"
        return self.fn(x, **kwargs) + x

In [ ]:
show_doc(Residual.forward)

<h4 id="Residual.forward" class="doc_header"><code>Residual.forward</code><a href="__main__.py#L10" class="source_link" style="float:right">[source]</a></h4>

> <code>Residual.forward</code>(**`x`**:`tensor`, **\*\*`kwargs`**)

Applies `self.fn` to x

**Function Arguments**:
* `x` (`torch.tensor`): An input
* `**kwargs`: kwargs for `self.fn`


In [ ]:
#export
class PreNorm(Module):
    def __init__(
        self, 
        dim:int, # LayerNorm dimension
        fn:callable # Residual function
    ):
        "Layer norm that is applied before calling a residual function"
        self.norm = nn.LayerNorm(dim)
        self.fn = fn

    def forward(
        self, 
        x:torch.tensor, # An input
        **kwargs # kwargs for `self.fn`
    ):
        "Applies `self.fn` to the output of `self.norm(x)`"
        return self.fn(self.norm(x), **kwargs)

In [ ]:
show_doc(PreNorm.forward)

<h4 id="PreNorm.forward" class="doc_header"><code>PreNorm.forward</code><a href="__main__.py#L12" class="source_link" style="float:right">[source]</a></h4>

> <code>PreNorm.forward</code>(**`x`**:`tensor`, **\*\*`kwargs`**)

Applies `self.fn` to the output of `self.norm(x)`

**Function Arguments**:
* `x` (`torch.tensor`): An input
* `**kwargs`: kwargs for `self.fn`


## Attention Layers

In [ ]:
#export
class GEGLU(Module):
    "GLU variation introduced in GLU Variants Improve Transformer"
    def forward(
        self, 
        x:torch.tensor, # An input
    ):
        "Chunks `x` into 2, applies `F.gelu` to chunks, and returns product with `x`"
        x, gates = x.chunk(2, dim = -1)
        return x * F.gelu(gates)

In [ ]:
show_doc(GEGLU.forward)

<h4 id="GEGLU.forward" class="doc_header"><code>GEGLU.forward</code><a href="__main__.py#L4" class="source_link" style="float:right">[source]</a></h4>

> <code>GEGLU.forward</code>(**`x`**:`tensor`)

Chunks `x` into 2, applies `F.gelu` to chunks, and returns product with `x`

**Function Arguments**:
* `x` (`torch.tensor`): An input


In [ ]:
#export
class FeedForward(nn.Sequential):
    def __init__(
        self, 
        dim:int, # Linear layer dimension 
        mult:int = 4, # Multiplier for `dim` 
        dropout:(int,float) = 0., # Dropout probability
    ):
        "Feed-Forward Layer"
        layers = [
            nn.Linear(dim, dim * mult * 2),
            GEGLU(),
            nn.Dropout(dropout),
            nn.Linear(dim * mult, dim)
        ]
        super().__init__(*layers)

    def forward(
        self, 
        x:torch.tensor, # An input
    ):
        "Applies x to a `Linear` -> `GEGLU` -> `Dropout` -> `Linear` group"
        return super().forward(x)

In [ ]:
show_doc(FeedForward.forward)

<h4 id="FeedForward.forward" class="doc_header"><code>FeedForward.forward</code><a href="__main__.py#L18" class="source_link" style="float:right">[source]</a></h4>

> <code>FeedForward.forward</code>(**`x`**:`tensor`)

Applies x to a `Linear` -> `GEGLU` -> `Dropout` -> `Linear` group

**Function Arguments**:
* `x` (`torch.tensor`): An input


In [ ]:
#export
class Attention(Module):
    def __init__(
        self,
        dim:int, # Dimension for the Linear groups
        heads:int = 8, # Number of attention heads
        dim_head:int = 16, # Dimension of the attention heads
        dropout:float = 0. # Dropout probability
    ):
        "A basic attention wrt module"
        inner_dim = dim_head * heads
        self.heads = heads
        self.scale = dim_head ** -0.5

        self.to_qkv = nn.Linear(dim, inner_dim * 3, bias = False)
        self.to_out = nn.Linear(inner_dim, dim)

        self.dropout = nn.Dropout(dropout)

    def forward(
        self, 
        x:torch.tensor # An input
    ):
        "Applies attention to `x`"
        h = self.heads
        q, k, v = self.to_qkv(x).chunk(3, dim = -1)
        q, k, v = map(lambda t: rearrange(t, 'b n (h d) -> b h n d', h = h), (q, k, v))
        sim = einsum('b h i d, b h j d -> b h i j', q, k) * self.scale
        attn = sim.softmax(dim = -1)
        out = einsum('b h i j, b h j d -> b h i d', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)', h = h)
        return self.to_out(out)

In [ ]:
show_doc(Attention.forward)

<h4 id="Attention.forward" class="doc_header"><code>Attention.forward</code><a href="__main__.py#L20" class="source_link" style="float:right">[source]</a></h4>

> <code>Attention.forward</code>(**`x`**:`tensor`)

Applies attention to `x`

**Function Arguments**:
* `x` (`torch.tensor`): An input


In [ ]:
#export
mk_class('AttentionType', **{'Column':'col', 'ColumnRow':'colrow'},
        doc="All possible attention types, with typo-proofing and auto-complete")

In [ ]:
show_doc(AttentionType, title_level=3)

<h3 id="AttentionType" class="doc_header"><code>class</code> <code>AttentionType</code><a href="" class="source_link" style="float:right">[source]</a></h3>

> <code>AttentionType</code>(**\*`args`**, **\*\*`kwargs`**)

All possible attention types, with typo-proofing and auto-complete



In [ ]:
#export
class RowColAttention(Module):
    def __init__(
        self, 
        num_tokens:int, # Size of the categorical embeddings
        dim:int, # Dimension of the two Embedding layers
        nfeats:int, # Number of continuous features
        depth:int, # The number of attention modules to generate 
        heads:int, # Number of attention heads 
        dim_head:int = 16, # Dimension of the attention heads
        attn_dropout:float = 0., # Dropout probability in the attention module
        ff_dropout:float = 0., # Dropout probability for the feed forward layers
        style:AttentionType = AttentionType.Column # Attention style
    ):
        "A small Transformer that generates attention based on a row and column"
        self.embeds = nn.Embedding(num_tokens, dim)
        self.layers = nn.ModuleList([])
        self.mask_embed =  nn.Embedding(nfeats, dim)
        self.style = style
        for _ in range(depth):
            if self.style == 'colrow':
                self.layers.append(nn.ModuleList([
                    PreNorm(dim, Residual(Attention(dim, heads = heads, dim_head = dim_head, dropout = attn_dropout))),
                    PreNorm(dim, Residual(FeedForward(dim, dropout = ff_dropout))),
                    PreNorm(dim*nfeats, Residual(Attention(dim*nfeats, heads = heads, dim_head = 64, dropout = attn_dropout))),
                    PreNorm(dim*nfeats, Residual(FeedForward(dim*nfeats, dropout = ff_dropout))),
                ]))
            elif self.style == 'col':
                self.layers.append(nn.ModuleList([
                    PreNorm(dim*nfeats, Residual(Attention(dim*nfeats, heads = heads, dim_head = 64, dropout = attn_dropout))),
                    PreNorm(dim*nfeats, Residual(FeedForward(dim*nfeats, dropout = ff_dropout))),
                ]))
            else:
                raise NotImplementedError(f'Attention style {style} is currently not supported, please choose a valid `AttentionType`')

    def forward(
        self, 
        x, # Categorical inputs
        x_cont=None, # Continuous inputs
        mask = None # Mask of our y's
    ):
        "Applies an attention mechanism on inputs"
        if x_cont is not None:
            x = torch.cat((x,x_cont),dim=1)
        _, n, _ = x.shape
        if self.style == 'colrow':
            for attn1, ff1, attn2, ff2 in self.layers: 
                x = attn1(x)
                x = ff1(x)
                x = rearrange(x, 'b n d -> 1 b (n d)')
                x = attn2(x)
                x = ff2(x)
                x = rearrange(x, '1 b (n d) -> b n d', n = n)
        else:
             for attn1, ff1 in self.layers:
                x = rearrange(x, 'b n d -> 1 b (n d)')
                x = attn1(x)
                x = ff1(x)
                x = rearrange(x, '1 b (n d) -> b n d', n = n)
        return x

In [ ]:
show_doc(RowColAttention.forward)

<h4 id="RowColAttention.forward" class="doc_header"><code>RowColAttention.forward</code><a href="__main__.py#L36" class="source_link" style="float:right">[source]</a></h4>

> <code>RowColAttention.forward</code>(**`x`**, **`x_cont`**=*`None`*, **`mask`**=*`None`*)

Applies an attention mechanism on inputs

**Function Arguments**:
* `x`: Categorical inputs
* `x_cont`: Continuous inputs


In [ ]:
#export
class Transformer(Module):
    def __init__(
        self, 
        num_tokens:int, # Size of the categorical embeddings in the `Attention` layer
        dim:int, # Dimension of the two Embedding layers in the `Attention` layer
        depth:int, # The number of attention modules to generate in the `Attention` layer
        heads:int, # Number of attention heads in the `Attention` layer
        dim_head:int = 16, # Dimension of the attention heads in the `Attention` layer
        attn_dropout:float = 0., # Dropout probability in the `Attention` layer
        ff_dropout:float = 0., # Dropout probability for the feed forward layers
    ):
        "A basic feed forward Transformer module with attention"
        self.layers = nn.ModuleList([])


        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                PreNorm(dim, Residual(Attention(dim, heads = heads, dim_head = dim_head, dropout = attn_dropout))),
                PreNorm(dim, Residual(FeedForward(dim, dropout = ff_dropout))),
            ]))

    def forward(
        self, 
        x, # Categorical inputs
        x_cont=None, # Continuous inputs
    ):
        "Applies attention to inputs"
        if x_cont is not None:
            x = torch.cat((x,x_cont),dim=1)
        for attn, ff in self.layers:
            x = attn(x)
            x = ff(x)
        return x

In [ ]:
show_doc(Transformer.forward)

<h4 id="Transformer.forward" class="doc_header"><code>Transformer.forward</code><a href="__main__.py#L23" class="source_link" style="float:right">[source]</a></h4>

> <code>Transformer.forward</code>(**`x`**, **`x_cont`**=*`None`*)

Applies attention to inputs

**Function Arguments**:
* `x`: Categorical inputs
* `x_cont`: Continuous inputs


In [ ]:
#export
class MLP(Module):
    def __init__(
        self, 
        dims:list, # A list of dimensions for the module
        act = None # An activation function, such as `GEGLU`
    ):
        "A basic multi-layer perceptron module"
        dims_pairs = list(zip(dims[:-1], dims[1:]))
        layers = []
        for ind, (dim_in, dim_out) in enumerate(dims_pairs):
            is_last = ind >= (len(dims) - 1)
            linear = nn.Linear(dim_in, dim_out)
            layers.append(linear)

            if is_last:
                continue
            if act is not None:
                layers.append(act)

        self.mlp = nn.Sequential(*layers)

    def forward(
        self, 
        x:torch.tensor # An input
    ):
        "Applies mlp on `x`"
        return self.mlp(x)

In [ ]:
show_doc(MLP.forward)

<h4 id="MLP.forward" class="doc_header"><code>MLP.forward</code><a href="__main__.py#L23" class="source_link" style="float:right">[source]</a></h4>

> <code>MLP.forward</code>(**`x`**:`tensor`)

Applies mlp on `x`

**Function Arguments**:
* `x` (`torch.tensor`): An input


In [ ]:
#export
class SimpleMLP(nn.Sequential):
    def __init__(
        self,
        dims:list # A list of three dimensions for our MLP module
    ):
        "Simplified multi-layer perceptron"
        layers = [
            nn.Linear(dims[0], dims[1]),
            nn.ReLU(),
            nn.Linear(dims[1], dims[2])
        ]
        super().__init__(*layers)
    
    def forward(
        self,
        x:torch.tensor # A tensor input
    ):
        "Applies simplified MLP on `x`"
        if len(x.shape) == 1: 
            x = x.view(x.shape(0), -1)
        return super().forward(x)

In [ ]:
show_doc(SimpleMLP.forward)

<h4 id="SimpleMLP.forward" class="doc_header"><code>SimpleMLP.forward</code><a href="__main__.py#L15" class="source_link" style="float:right">[source]</a></h4>

> <code>SimpleMLP.forward</code>(**`x`**:`tensor`)

Applies simplified MLP on `x`

**Function Arguments**:
* `x` (`torch.tensor`): A tensor input


In [ ]:
#export
from typing import Tuple

In [ ]:
#export
class TabAttention(nn.Module):
    def __init__(
        self,
        categories:list, # List of categorical cardinalities
        num_continuous:int, # Number of continuous variables
        dim:int, # MLP dimension
        depth:int, # Transformer depth
        heads:int, # Number of attention heads
        dim_head:int = 16, # Size of the attention head
        dim_out:int = 1, # Size of the last linear layer
        mlp_hidden_mults:Tuple[int] = (4, 2), # Multipliers for the MLP hidden layers
        mlp_act = None, # Activation class of the MLP module, such as `GEGLU`
        num_special_tokens:int = 1, # Number of special tokens for the categories
        attn_dropout:float = 0., # Dropout probability for the attention module
        ff_dropout:float = 0., # Dropout probability for the feed forward layers
        lastmlp_dropout:float = 0., # Dropout probability for the final MLP group
        cont_embeddings:str = 'MLP', # Type of embeddings for the continouous variables, only `MLP` is available. If `None` no attention is applied
        attention_style:AttentionType = AttentionType.Column # Attention style
    ):
        assert all(map(lambda n: n > 0, categories)), 'number of each category must be positive'

        # categories related calculations
        self.num_categories = len(categories)
        self.num_unique_categories = sum(categories)

        # create category embeddings table

        self.num_special_tokens = num_special_tokens
        self.total_tokens = self.num_unique_categories + num_special_tokens

        # for automatically offsetting unique category ids to the correct position in the categories embedding table
        categories_offset = F.pad(torch.tensor(list(categories)), (1, 0), value = num_special_tokens)
        categories_offset = categories_offset.cumsum(dim = -1)[:-1]
        
        self.register_buffer('categories_offset', categories_offset)


        self.norm = nn.LayerNorm(num_continuous)
        self.num_continuous = num_continuous
        self.dim = dim
        self.cont_embeddings = cont_embeddings
        self.attentiontype = attentiontype

        if self.cont_embeddings == 'MLP':
            self.simple_MLP = nn.ModuleList([simple_MLP([1,100,self.dim]) for _ in range(self.num_continuous)])
            input_size = (dim * self.num_categories)  + (dim * num_continuous)
            nfeats = self.num_categories + num_continuous
        else:
            print('Continous features are not passed through attention')
            input_size = (dim * self.num_categories) + num_continuous
            nfeats = self.num_categories 

        # transformer
        if attention_style == 'col':
            self.transformer = Transformer(
                num_tokens = self.total_tokens,
                dim = dim,
                depth = depth,
                heads = heads,
                dim_head = dim_head,
                attn_dropout = attn_dropout,
                ff_dropout = ff_dropout
            )
        elif attention_style == 'colrow':
            self.transformer = RowColTransformer(
                num_tokens = self.total_tokens,
                dim = dim,
                nfeats= nfeats,
                depth = depth,
                heads = heads,
                dim_head = dim_head,
                attn_dropout = attn_dropout,
                ff_dropout = ff_dropout,
                style = attentiontype
            )

        l = input_size // 8
        hidden_dimensions = list(map(lambda t: l * t, mlp_hidden_mults))
        all_dimensions = [input_size, *hidden_dimensions, dim_out]
        
        self.mlp = MLP(all_dimensions, act = mlp_act)
        self.embeds = nn.Embedding(self.total_tokens, self.dim) #.to(device)

        cat_mask_offset = F.pad(torch.Tensor(self.num_categories).fill_(2).type(torch.int8), (1, 0), value = 0) 
        cat_mask_offset = cat_mask_offset.cumsum(dim = -1)[:-1]

        con_mask_offset = F.pad(torch.Tensor(self.num_continuous).fill_(2).type(torch.int8), (1, 0), value = 0) 
        con_mask_offset = con_mask_offset.cumsum(dim = -1)[:-1]

        self.register_buffer('cat_mask_offset', cat_mask_offset)
        self.register_buffer('con_mask_offset', con_mask_offset)

        self.mask_embeds_cat = nn.Embedding(self.num_categories*2, self.dim)
        self.mask_embeds_cont = nn.Embedding(self.num_continuous*2, self.dim)

    def forward(
        self, 
        x_categ:torch.tensor, # Categorical inputs
        x_cont:torch.tensor, # Continous inputs
        x_categ_enc:torch.tensor, # Encoded categorical inputs via `embed_data_mask`
        x_cont_enc:torch.tensor # Encoded continuous inputs via `embed_data_mask`
    ):
        "Feed input through Tabular Attention"
        device = x_categ.device
        if self.attentiontype == 'justmlp':
            if x_categ.shape[-1] > 0:
                flat_categ = x_categ.flatten(1).to(device)
                x = torch.cat((flat_categ, x_cont.flatten(1).to(device)), dim = -1)
            else:
                x = x_cont.clone()
        else:
            if self.cont_embeddings == 'MLP':
                x = self.transformer(x_categ_enc,x_cont_enc.to(device))
            else:
                if x_categ.shape[-1] <= 0:
                    x = x_cont.clone()
                else: 
                    flat_categ = self.transformer(x_categ_enc).flatten(1)
                    x = torch.cat((flat_categ, x_cont), dim = -1)                    
        flat_x = x.flatten(1)
        return self.mlp(flat_x)

In [ ]:
show_doc(TabAttention.forward)

<h4 id="TabAttention.forward" class="doc_header"><code>TabAttention.forward</code><a href="__main__.py#L97" class="source_link" style="float:right">[source]</a></h4>

> <code>TabAttention.forward</code>(**`x_categ`**:`tensor`, **`x_cont`**:`tensor`, **`x_categ_enc`**:`tensor`, **`x_cont_enc`**:`tensor`)

Feed input through Tabular Attention

**Function Arguments**:
* `x_categ` (`torch.tensor`): Categorical inputs
* `x_cont` (`torch.tensor`): Continous inputs
* `x_categ_enc` (`torch.tensor`): Encoded categorical inputs via `embed_data_mask`
* `x_cont_enc` (`torch.tensor`): Encoded continuous inputs via `embed_data_mask`
